# Semantic Kernel

Trong ví dụ mã này, bạn sẽ sử dụng [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework để tạo một tác nhân cơ bản.

Mục tiêu của ví dụ này là giới thiệu các bước mà chúng ta sẽ sử dụng sau này trong các ví dụ mã bổ sung khi triển khai các mẫu tác nhân khác nhau.


## Nhập các Gói Python Cần Thiết


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Tạo Client

Trong ví dụ này, chúng ta sẽ sử dụng [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) để truy cập vào LLM.

`ai_model_id` được định nghĩa là `gpt-4o-mini`. Hãy thử thay đổi mô hình sang một mô hình khác có sẵn trên marketplace của GitHub Models để xem các kết quả khác nhau.

Để sử dụng `Azure Inference SDK`, được sử dụng cho `base_url` của GitHub Models, chúng ta sẽ sử dụng connector `OpenAIChatCompletion` trong Semantic Kernel. Ngoài ra, còn có các [connector khác](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) có sẵn để sử dụng Semantic Kernel với các nhà cung cấp mô hình khác.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Tạo Agent

Dưới đây, chúng ta sẽ tạo Agent có tên là `TravelAgent`.

Trong ví dụ này, chúng ta sử dụng các hướng dẫn rất đơn giản. Bạn có thể thay đổi các hướng dẫn này để xem cách Agent phản hồi khác nhau.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Chạy Agent

Bây giờ chúng ta có thể chạy Agent bằng cách định nghĩa một luồng thuộc loại `ChatHistoryAgentThread`. Bất kỳ tin nhắn hệ thống nào cần thiết đều được cung cấp cho tham số `messages` của `invoke_stream` trong agent.

Sau khi các tin nhắn này được định nghĩa, chúng ta tạo một `user_inputs`, đây sẽ là nội dung mà người dùng gửi đến agent. Trong trường hợp này, chúng ta đã đặt tin nhắn này là `Plan me a sunny vacation`.

Bạn có thể thoải mái thay đổi tin nhắn này để xem cách agent phản hồi khác nhau.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
